### Data mining

In [1]:
import pandas as pd
import thinkplot
import numpy as np
import statsmodels.formula.api as smf

Read in data

In [31]:
df_raw = pd.read_csv('../Data/usa_00002.csv')
df_raw.head()

,YEAR,DATANUM,SERIAL,CBSERIAL,HHWT,REGION,STATEICP,STATEFIP,COUNTYICP,COUNTYFIP,...,DIFFHEAR,PWSTATE2,PWCOUNTY,PWMET13,PWTYPE,PWPUMA00,TRANWORK,TRANTIME,DEPARTS,ARRIVES
0,2017,1,1,2017000000016,206,32,41,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2017,1,2,2017000000031,45,32,41,1,0,0,...,1,1,117,13820,5,1200,10,50,642,734
2,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,13,0,0,5,1700,10,45,1805,1849
3,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
work_force = df_raw[df_raw.EMPSTAT == 1] # Consider all employed people

Remove NaNs

In [5]:
work_force = work_force.dropna()
work_force.head()

,YEAR,DATANUM,SERIAL,CBSERIAL,HHWT,REGION,STATEICP,STATEFIP,COUNTYICP,COUNTYFIP,...,DIFFHEAR,PWSTATE2,PWCOUNTY,PWMET13,PWTYPE,PWPUMA00,TRANWORK,TRANTIME,DEPARTS,ARRIVES
1,2017,1,2,2017000000031,45,32,41,1,0,0,...,1,1,117,13820,5,1200,10,50,642,734
2,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,13,0,0,5,1700,10,45,1805,1849
5,2017,1,4,2017000000158,19,32,41,1,0,0,...,1,1,0,0,9,2500,10,25,717,744
10,2017,1,5,2017000000159,21,32,41,1,0,0,...,1,1,0,0,9,290,10,60,602,704
11,2017,1,5,2017000000159,21,32,41,1,0,0,...,1,1,0,0,9,290,10,85,702,829


Recode `SEX` to be 0 for female and 1 for male.

In [6]:
work_force['SEX'].replace(2, 0, inplace=True)
work_force['SEX'].value_counts()

1    775971
0    713015
Name: SEX, dtype: int64

Find hourly income using `UHRSWORK`, Usual hours worked per week, and `WKSWORK2`, Weeks worked last year (intervalled).

In [7]:
work_force['UHRSWORK'].replace([0], np.nan, inplace=True)
work_force['WKSWORK2'].replace([0], np.nan, inplace=True)
work_force['INCWAGE'].replace([0, 999999, 999998], np.nan, inplace=True)

In [8]:
work_force['HRLY_INCWAGE'] = work_force['INCWAGE']/(work_force['UHRSWORK'] * work_force['WKSWORK2'])

In [9]:
work_force['HRLY_INCWAGE'].describe()

count    1.405825e+06
mean     2.439490e+02
std      1.181061e+03
min      6.734007e-03
25%      1.041667e+02
50%      1.666667e+02
75%      2.777778e+02
max      6.380000e+05
Name: HRLY_INCWAGE, dtype: float64

### Predictive powers

#### Find variables that impact income the most

In [32]:
t = []
for name in work_force.columns:
    try:
        if work_force[name].var() < 1e-7:
            continue
        
        formula = 'INCWAGE ~ ' + name
        model = smf.ols(formula, data=work_force)
        if model.nobs < len(work_force)/2:
            continue
            
        results = model.fit()
    except (ValueError, TypeError):
        continue
        
    t.append((results.rsquared, name))

In [33]:
t.sort(reverse=True)
for mse, name in t[:-1]:
    print(name, mse)

INCWAGE 1.0
INCEARN 0.9643539944174833
INCTOT 0.902729536337456
OCCSCORE 0.18286323196191778
UHRSWORK 0.12402149901584703
EDUCD 0.11759843360336752
SEI 0.1172720385586764
EDUC 0.11280918853939792
DEGFIELDD 0.09497986675277326
DEGFIELD 0.094931220832427
OCC2010 0.06949291741871366
OCC 0.06943137181608505
OCC1950 0.06657706000955743
WKSWORK2 0.05517219681764918
MARST 0.05246340203610922
YRMARR 0.04463904871535018
HRLY_INCWAGE 0.044595057096083024
BIRTHYR 0.03626155877132242
AGE 0.0362615587713222
RELATE 0.02733336528673147
RELATED 0.027213563708235577
SEX 0.02718355832449204
PERNUM 0.025688389310253523
SCHOOL 0.02567314704248469
GRADEATTD 0.023518076071295746
GRADEATT 0.02316318631521619
SCHLTYPE 0.021020686371472586
FERTYR 0.019072713485713932
ELDCH 0.013958445991431367
YNGCH 0.013634390687859876
NCHILD 0.010229289721466195
DEPARTS 0.009578271959497786
TRANTIME 0.009237916397048518
DEGFIELD2 0.009236210797644029
DEGFIELD2D 0.009236074697404506
GQ 0.008462723206650424
PWTYPE 0.0082707506

#### Income ~ variables

In [21]:
# Build formula
formula = 'HRLY_INCWAGE ~ '
ignores = ['HRLY_INCWAGE', 'SEX', 'UHRSWORK', 'WKSWORK2', 'INCWAGE', 'INCEARN', 'INCTOT', 'OCCSCORE', 'SEI']
for i in range(len(work_force.columns)):
    name = work_force.columns[i]
    try:
        if work_force[name].var() < 1e-7 or name in ignores:
            continue

        formula += name + ' + '

    except (ValueError, TypeError):
        continue

In [22]:
formula = formula[:-3]
formula

'HRLY_INCWAGE ~ SERIAL + CBSERIAL + HHWT + REGION + STATEICP + STATEFIP + COUNTYICP + COUNTYFIP + METRO + MET2013 + MET2013ERR + CITY + CITYERR + CITYPOP + PUMA + CPUMA0010 + HOMELAND + GQ + FARM + NFAMS + PERNUM + PERWT + NCHILD + NCHLT5 + ELDCH + YNGCH + RELATE + RELATED + AGE + MARST + BIRTHYR + YRMARR + FERTYR + RACE + RACED + HISPAN + HISPAND + CITIZEN + YRSUSA1 + SPEAKENG + SCHOOL + EDUC + EDUCD + GRADEATT + GRADEATTD + SCHLTYPE + DEGFIELD + DEGFIELDD + DEGFIELD2 + DEGFIELD2D + EMPSTATD + OCC + OCC1950 + OCC2010 + IND + IND1990 + CLASSWKR + CLASSWKRD + FTOTINC + VETDISAB + DIFFREM + DIFFPHYS + DIFFMOB + DIFFCARE + DIFFSENS + DIFFEYE + DIFFHEAR + PWSTATE2 + PWCOUNTY + PWMET13 + PWTYPE + PWPUMA00 + TRANWORK + TRANTIME + DEPARTS + ARRIVES'

In [27]:
model = smf.ols(formula, data=work_force)
results = model.fit()
results.rsquared

0.014334919160695181

#### Income ~ variables + sex

In [28]:
formula_sex = formula + ' + SEX'

In [30]:
model = smf.ols(formula_sex, data=work_force)
results = model.fit()
results.rsquared

0.014836905381526488

### Analysis into some variables